In [1]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 MB 10.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.5-py2.py3-none-any.whl size=317747883 sha256=de08859c269a8c0a677132479fff61dc9b6d1530ff601ea110420e7128787f0a
  Stored in directory: /Users/allullaswarantej/Library/Caches/pip/wheels/af/de/65/358d44d6c11fd5795dcbda700ff8b7ab3cbcdfde7f4fb1d9fb
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession

# initialize SparkSession
spark = SparkSession.builder \
    .appName("ETL Pipeline") \
    .getOrCreate()

25/03/26 15:42:15 WARN Utils: Your hostname, Allullas-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.102 instead (on interface en0)
25/03/26 15:42:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/26 15:42:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [17]:
# load the CSV file into a Spark DataFrame
file_path = "/Users/allullaswarantej/Downloads/temperature.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# display the schema and preview the data
df.printSchema()
df.show(5)

root
 |-- ObjectId: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- ISO2: string (nullable = true)
 |-- ISO3: string (nullable = true)
 |-- F1961: double (nullable = true)
 |-- F1962: double (nullable = true)
 |-- F1963: double (nullable = true)
 |-- F1964: double (nullable = true)
 |-- F1965: double (nullable = true)
 |-- F1966: double (nullable = true)
 |-- F1967: double (nullable = true)
 |-- F1968: double (nullable = true)
 |-- F1969: double (nullable = true)
 |-- F1970: double (nullable = true)
 |-- F1971: double (nullable = true)
 |-- F1972: double (nullable = true)
 |-- F1973: double (nullable = true)
 |-- F1974: double (nullable = true)
 |-- F1975: double (nullable = true)
 |-- F1976: double (nullable = true)
 |-- F1977: double (nullable = true)
 |-- F1978: double (nullable = true)
 |-- F1979: double (nullable = true)
 |-- F1980: double (nullable = true)
 |-- F1981: double (nullable = true)
 |-- F1982: double (nullable = true)
 |-- F1983: double (nullable 

25/03/26 15:46:51 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+--------------------+----+----+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+-----+------+-----+------+------+------+-----+------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|ObjectId|             Country|ISO2|ISO3| F1961| F1962| F1963| F1964| F1965|F1966| F1967| F1968| F1969| F1970| F1971| F1972| F1973| F1974| F1975| F1976| F1977| F1978|F1979| F1980| F1981| F1982| F1983| F1984| F1985| F1986| F1987|F1988| F1989|F1990| F1991| F1992| F1993|F1994| F1995| F1996|F1997|F1998|F1999|F2000|F2001|F2002|F2003|F2004|F2005|F2006|F2007|F2008|F2009|F2010|F2011|F2012|F2013|F2014|F2015|F2016|F2017|F2018|F2019|F2020|F2021|F2022|
+--------+--------------------+----+----+------+------+------+------+------+-----+------+------+------+------+

In [18]:
# fill missing values for country codes
df = df.fillna({"ISO2": "Unknown"})

# drop rows where all temperature values are null
temperature_columns = [col for col in df.columns if col.startswith('F')]
df = df.dropna(subset=temperature_columns, how="all")

In [20]:
from pyspark.sql.functions import expr

# reshape temperature data to have 'Year' and 'Temperature' columns
df_pivot = df.selectExpr(
    "ObjectId", "Country", "ISO3",
    "stack(62, " + 
    ",".join([f"'F{1961 + i}', F{1961 + i}" for i in range(62)]) +
    ") as (Year, Temperature)"
)

# convert 'Year' column to integer
df_pivot = df_pivot.withColumn("Year", expr("int(substring(Year, 2, 4))"))
df_pivot.show(10)

+--------+--------------------+----+----+-----------+
|ObjectId|             Country|ISO3|Year|Temperature|
+--------+--------------------+----+----+-----------+
|       1|Afghanistan, Isla...| AFG|1961|     -0.113|
|       1|Afghanistan, Isla...| AFG|1962|     -0.164|
|       1|Afghanistan, Isla...| AFG|1963|      0.847|
|       1|Afghanistan, Isla...| AFG|1964|     -0.764|
|       1|Afghanistan, Isla...| AFG|1965|     -0.244|
|       1|Afghanistan, Isla...| AFG|1966|      0.226|
|       1|Afghanistan, Isla...| AFG|1967|     -0.371|
|       1|Afghanistan, Isla...| AFG|1968|     -0.423|
|       1|Afghanistan, Isla...| AFG|1969|     -0.539|
|       1|Afghanistan, Isla...| AFG|1970|      0.813|
+--------+--------------------+----+----+-----------+
only showing top 10 rows



In [27]:
output_path = "Users/allullaswarantej/Downloads/processed_temperature.parquet"
df_pivot.write.mode("overwrite").parquet(output_path)

In [28]:
# load the saved parquet file
processed_df = spark.read.parquet(output_path)
processed_df.show(5)

+--------+--------------------+----+----+-----------+
|ObjectId|             Country|ISO3|Year|Temperature|
+--------+--------------------+----+----+-----------+
|       1|Afghanistan, Isla...| AFG|1961|     -0.113|
|       1|Afghanistan, Isla...| AFG|1962|     -0.164|
|       1|Afghanistan, Isla...| AFG|1963|      0.847|
|       1|Afghanistan, Isla...| AFG|1964|     -0.764|
|       1|Afghanistan, Isla...| AFG|1965|     -0.244|
+--------+--------------------+----+----+-----------+
only showing top 5 rows

